In [ ]:
## Feature Extraction

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(os.path.join(basedir,'shape_predictor_68_face_landmarks.dat'))


In [ ]:
os.chdir=("/Users/Hyunjee/Desktop/AMLS_19-20_SN16075203/A2/celeba")

basedir = os.path.abspath(os.curdir)

images_dir = os.path.join(basedir,'img')
# for item in os.listdir(images_dir):
#     if not item.startswith('.') and os.path.isfile(os.path.join(images_dir, item)):
#         print(len(image_paths))
#         print(image_paths)



labels = os.path.join(basedir,'labels.csv')

#print(os.path.abspath(labels_filename))

In [ ]:

def extract_features_labels(images_dir):
    """
    This funtion extracts the landmarks features for all images in the folder 'dataset/celeba'.
    It also extracts the gender label for each image.
    :return:
        landmark_features:  an array containing 68 landmark points for each image in which a face was detected
        emotion_labels:      an array containing the gender label (smiling=0 and notsmiling=1) for each image in
                            which a face was detected
    """
    image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir) if not l.endswith('.DS_Store')]
    
    target_size = None
    labels_file = open(os.path.join(basedir, labels_filename), 'r')
    #labels_file = open(os.path.join(basedir, labels), 'r')
    
    lines = labels_file.readlines()
    emotion_labels = {line.split('.')[0] : int(line.split(',')[2]) for line in lines[1:]}
    #emotion_labels ={line.split('\t')[0] : int(line.split('\t')[3]) for line in lines[1:]}
    if os.path.isdir(images_dir):
        all_features = []
        all_labels = []
        for img_path in image_paths:
            file_name= img_path.split('/')[-1].split('.')[0]
            

            # load image
            img = image.img_to_array(
                image.load_img(img_path,
                               target_size=target_size,
                               interpolation='bicubic'))
            features, _ = run_dlib_shape(img)
            if features is not None:
                all_features.append(features)
                all_labels.append(emotion_labels[file_name])

    landmark_features = np.array(all_features)
    emotion_labels = (np.array(all_labels) + 1)/2 # simply converts the -1 into 0, so smiling=0 and not smiling=1
    
    return landmark_features, emotion_labels

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)


def run_dlib_shape(image):
    # in this function we load the image, detect the landmarks of the face, and then return the image and the landmarks
    # load the input image, resize it, and convert it to grayscale
    resized_image = image.astype('uint8')

    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    num_faces = len(rects)

    if num_faces == 0:
        return None, resized_image

    face_areas = np.zeros((1, num_faces))
    face_shapes = np.zeros((136, num_faces), dtype=np.int64)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        temp_shape = predictor(gray, rect)
        temp_shape = shape_to_np(temp_shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)],
        #   (x, y, w, h) = face_utils.rect_to_bb(rect)
        (x, y, w, h) = rect_to_bb(rect)
        face_shapes[:, i] = np.reshape(temp_shape, [136])
        face_areas[0, i] = w * h
    # find largest face and keep
    dlibout = np.reshape(np.transpose(face_shapes[:, np.argmax(face_areas)]), [68, 2])

    return dlibout, resized_image



In [ ]:
s_train_images_dir = os.path.join(basedir,'train','smiling')
ns_train_images_dir = os.path.join(basedir,'train','NotSmiling')

s_test_images_dir = os.path.join(basedir,'test','smiling')
ns_test_images_dir = os.path.join(basedir,'test','NotSmiling')

s_valid_images_dir = os.path.join(basedir,'validation','smiling')
ns_valid_images_dir = os.path.join(basedir,'validation','NotSmiling')

s_train_feature, s_train_label = extract_features_labels(s_train_images_dir)
ns_train_feature, ns_train_label = extract_features_labels(ns_train_images_dir)

s_test_feature, s_test_label = extract_features_labels(s_test_images_dir)
ns_test_feature, ns_test_label = extract_features_labels(ns_test_images_dir)

s_valid_feature, s_valid_label = extract_features_labels(s_valid_images_dir)
ns_valid_feature, ns_valid_label = extract_features_labels(ns_valid_images_dir)



In [ ]:
#os.mkdir("/Users/Hyunjee/Desktop/AMLS_19-20_SN16075203/A2/celeba/cropped")
os.chdir=("/Users/Hyunjee/Desktop/AMLS_19-20_SN16075203/A2/celeba/cropped")

In [ ]:
def crop(images_dir, shape):

    #img =  cv2.imread(images_dir)
#     cropped = img[230:230+58, 175:175+58]    
#     cropped = cv2.rectangle(image,(shape[36][0], shape[36][1]), (shape[45][0],shape[33][1]), (255,0,0), 1)
    i=0
    for f in glob.glob(os.path.join(images_dir, "*.jpg")):
        print("Processing file: {}".format(f))
        img = cv2.imread(f)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    
    # Get the landmarks/parts for the face in box d.
        i += 1
            # The next lines of code just get the coordinates for the mouth
            # and crop the mouth from the image.This part can probably be optimised
            # by taking only the outer most points.
        #for shapes in shape:
#         for x in range(48,68):
#             xmouthpoints = shape.part(x).x 
#             ymouthpoints = shape.part(x).y

        import itertools
        x,y=itertools.izip(*shades)
          
            
        maxx = max(xmouthpoints)
        minx = min(xmouthpoints)
        maxy = max(ymouthpoints)
        miny = min(ymouthpoints) 

            # to show the mouth properly pad both sides
        pad = 10
            # basename gets the name of the file with it's extension
            # splitext splits the extension and the filename
            # This does not consider the condition when there are multiple faces in each image.
            # if there are then it just overwrites each image and show only the last image.
        filename = os.path.splitext(os.path.basename(f))[0]


        crop_image = img[miny-pad:maxy+pad,minx-pad:maxx+pad]
            #cv2.imshow('mouth',crop_image)
            # The mouth images are saved in the format 'mouth1.jpg, mouth2.jpg,..
            # Change the folder if you want to. They are stored in the current directory
        cv2.imwrite(filename+'.jpg',crop_image)  
        
    return cropped